In [ ]:
import os
import datetime as dt
import pandas as pd
import time

In [ ]:
AllPOR = pd.read_excel(r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\JoannePOR\POR Waterfall_Yr 2020.xlsx',
                       usecols='A,C,E:H,J,K:CJ',sheet_name="W09 POR Data")

In [ ]:
def convert (PORDATA):
    Dates = PORDATA.columns[7:].values
    PORDATA['Attribute'] = 'N/A'
    col = 7
    new_por = pd.DataFrame()
    for date in Dates:
        PORDATA['Attribute'] = date
        PORDATA['Value'] = PORDATA.iloc[:,col]
        new_por = new_por.append(PORDATA)
        col = col + 1
    final_por = new_por.iloc[:,list(range(7)) + [-2,-1]]
    return final_por

In [ ]:
AllPOR.iloc[:,7:] = AllPOR.iloc[:,7:].fillna(0)

In [ ]:
AllPOR.tail(10)

In [ ]:
convert(AllPOR).to_csv('2020W09.csv',index=False)

# Files

**J POR**

In [ ]:
path = os.getcwd()
files = os.listdir(
    r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\JoannePOR\Converted\2019POR')
os.chdir(r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\JoannePOR\Converted\2019POR')

In [ ]:
files_csv = [f for f in files if f[0:4] == '2019']

In [ ]:
df = pd.DataFrame()

for f in files_csv:
    data = pd.read_csv(f)
    df = df.append(data)

df.head()

**List all files in directory**

In [ ]:
path = os.getcwd()
files = os.listdir(
    r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\2018POR')
os.chdir(r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\2018POR')

**Select all XXXX POR files (Change the code XXXX for different year)**

In [ ]:
files_xlsx = [f for f in files if f[4:8] == '2018']

**Merge all XXXX POR files into one**

In [ ]:
df = pd.DataFrame()

for f in files_xlsx:
    data = pd.read_excel(f)
    df = df.append(data)

df.head()

In [ ]:
df['Time'] = pd.to_datetime(df['Attribute'])

df['YYYYWW'] = df['Time'].apply(
    lambda x: str(x.isocalendar()[0]) + str(x.isocalendar()[1]).zfill(2))
df.head()

**Get region base on target location base on original excel file**

In [ ]:
dfreg = pd.read_excel(
    r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\Others\MJ.xlsm',
    sheet_name='2Region_Ref',
    na_filter=False,
    usecols='B:C')

# Merge region with XXXX POR files base on target location and convert date to int

In [ ]:
dfmerge = pd.merge(df, dfreg, on='Target Location')

dfmerge.head()

In [ ]:
dfmerge.drop(columns=['Attribute'])
dfmerge['Time'] = (dfmerge['Time'] - dt.datetime(1899, 12, 30)).dt.days
dfmerge = dfmerge.drop(columns=['Attribute'])

In [ ]:
dfmerge.head()

# Create Primary Key and Foreign Key

In [ ]:
dfmerge[[
    'Target Location', 'MPA', 'Planning Part', 'Time',
    'Current Cycle on Display'
]] = dfmerge[[
    'Target Location', 'MPA', 'Planning Part', 'Time',
    'Current Cycle on Display'
]].astype('str')

dfmerge['PK'] = dfmerge['Target Location'] + dfmerge['MPA'] + \
    dfmerge['Planning Part'] + dfmerge['Time'] + \
    dfmerge['Current Cycle on Display']
dfmerge['FK'] = dfmerge['Region'] + dfmerge['Planning Part'] + \
    dfmerge['Time'] + dfmerge['Current Cycle on Display']

In [ ]:
dfmerge.head()

# Finding Duplicates on primary key (IF ANY)

In [ ]:
duplicateRowsDF = dfmerge[dfmerge.duplicated(['PK'])]

duplicateRowsDF

# Group By Primary Key

In [ ]:
dfmerge2 = dfmerge.groupby('PK', as_index=False)['Value'].sum()

dfmerge2

# Drop Duplicates (IF ANY)

In [ ]:
dropdf = dfmerge.drop_duplicates(subset='PK', keep='first')

In [ ]:
dropdf

In [ ]:
dfdropvalue = dropdf.drop(columns='Value')

In [ ]:
dfdropvalue.head()

# Merge on Primary Key and remove word "group" from Planning Part Group

In [ ]:
dffinal = pd.merge(dfmerge2, dfdropvalue, on='PK')
dffinal['Planning Part Group'] = dffinal['Planning Part Group'].str.rstrip(
    'GROUP')
dffinal.head()

# Rearrange Data and Output

In [ ]:
dffinalarr = dffinal[[
    'PK', 'Current Cycle on Display', 'Product Line', 'Platform',
    'Planning Part Group', 'Target Location', 'Planning Part', 'MPA', 'Time',
    'Value', 'FK'
]]

In [ ]:
por_df = dffinal.rename(
    columns={
        "Current Cycle on Display": "Planning_Wk",
        "Product Line": "Product_Line",
        "Planning Part Group": "Program",
        "Target Location": "Target_Location",
        "Planning Part": "SKU",
        "Time": "WkDate",
        "Value": "Qty"
    })
por_df['QtyType'] = 'POR'
por_df.head()

In [ ]:
por_df.to_csv('2019_to_upload.csv', index=False)